In [1]:
import pandas as pd
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read the CSV into a Pandas DataFrame
df = pd.read_csv("./Resources/recipes.csv", low_memory=False)

# Print the first five rows of data to the screen
df.head()

,Unnamed: 0,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src
0,0,Apple-Cranberry Crostada,NaN,NaN,NaN,8,6 to 8 - servings,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,4.4,https://www.allrecipes.com/recipe/76931/apple-...,/Desserts/Fruit Desserts/Apple Dessert Recipes/,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...","Servings: 8, Yield: 6 to 8 - servings",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...
1,1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...
2,2,Sarah's Homemade Applesauce,10 mins,15 mins,25 mins,4,NaN,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,/Side Dish/Applesauce Recipes/,"Total Fat 0g 0%, Sodium 3mg 0%, Total Carbohyd...","Prep Time: 10 mins, Cook Time: 15 mins, Total ...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...
3,3,Apple Crisp,30 mins,45 mins,1 hrs 15 mins,12,1 9x13-inch pan,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,4.7,https://www.allrecipes.com/recipe/12409/apple-...,/Desserts/Crisps and Crumbles Recipes/Apple Cr...,"Total Fat 8g 11%, Saturated Fat 5g 25%, Choles...","Prep Time: 30 mins, Cook Time: 45 mins, Total ...",https://www.allrecipes.com/thmb/uAzhPOh86PfR-N...
4,4,Apple Pie Filling,20 mins,20 mins,2 hrs 40 mins,40,5 9-inch pies,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,4.7,https://www.allrecipes.com/recipe/12681/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 0g 0%, Sodium 61mg 3%, Total Carbohy...","Prep Time: 20 mins, Cook Time: 20 mins, Additi...",https://www.allrecipes.com/thmb/c0bbYaS1V_mTt_...


In [3]:
# Check null
df.isna().any()

Unnamed: 0      False
recipe_name     False
prep_time        True
cook_time        True
total_time       True
servings        False
yield            True
ingredients     False
directions      False
rating          False
url             False
cuisine_path    False
nutrition       False
timing          False
img_src         False
dtype: bool

In [4]:
# Dropup null
df.dropna(inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 631 entries, 1 to 1088
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    631 non-null    int64  
 1   recipe_name   631 non-null    object 
 2   prep_time     631 non-null    object 
 3   cook_time     631 non-null    object 
 4   total_time    631 non-null    object 
 5   servings      631 non-null    int64  
 6   yield         631 non-null    object 
 7   ingredients   631 non-null    object 
 8   directions    631 non-null    object 
 9   rating        631 non-null    float64
 10  url           631 non-null    object 
 11  cuisine_path  631 non-null    object 
 12  nutrition     631 non-null    object 
 13  timing        631 non-null    object 
 14  img_src       631 non-null    object 
dtypes: float64(1), int64(2), object(12)
memory usage: 78.9+ KB


In [6]:
# Check duplicated
df['recipe_name'].duplicated().value_counts()

False    557
True      74
Name: recipe_name, dtype: int64

In [7]:
# Dropout duplicated and double check
df_clean = df.drop_duplicates(subset=['recipe_name'])
df_clean['recipe_name'].duplicated().value_counts()

False    557
Name: recipe_name, dtype: int64

In [8]:
# Rename the column "Unnamed: 0" to "id" in the DataFrame
df_clean.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

# Display the first few rows of the modified DataFrame
df_clean.head()

,id,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src
1,1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...
3,3,Apple Crisp,30 mins,45 mins,1 hrs 15 mins,12,1 9x13-inch pan,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,4.7,https://www.allrecipes.com/recipe/12409/apple-...,/Desserts/Crisps and Crumbles Recipes/Apple Cr...,"Total Fat 8g 11%, Saturated Fat 5g 25%, Choles...","Prep Time: 30 mins, Cook Time: 45 mins, Total ...",https://www.allrecipes.com/thmb/uAzhPOh86PfR-N...
4,4,Apple Pie Filling,20 mins,20 mins,2 hrs 40 mins,40,5 9-inch pies,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,4.7,https://www.allrecipes.com/recipe/12681/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 0g 0%, Sodium 61mg 3%, Total Carbohy...","Prep Time: 20 mins, Cook Time: 20 mins, Additi...",https://www.allrecipes.com/thmb/c0bbYaS1V_mTt_...
7,7,Apple-Cranberry Crisp,25 mins,40 mins,1 hrs 5 mins,8,1 8-inch crisp,"2 pounds Granny Smith apples - peeled, cored a...",Preheat the oven to 375 degrees F (190 degrees...,4.6,https://www.allrecipes.com/recipe/41893/apple-...,/Desserts/Crisps and Crumbles Recipes/,"Total Fat 12g 15%, Saturated Fat 4g 22%, Chole...","Prep Time: 25 mins, Cook Time: 40 mins, Total ...",https://www.allrecipes.com/thmb/MjDDFmBWiEx9f1...
10,10,Apple Turnovers,30 mins,25 mins,55 mins,8,8 turnovers,"2 tablespoons lemon juice, 4 cups water, 4 Gr...",Combine the lemon and 4 cups water in a large ...,4.6,https://www.allrecipes.com/recipe/59124/apple-...,/Desserts/Fruit Desserts/Apple Dessert Recipes/,"Total Fat 26g 33%, Saturated Fat 8g 39%, Chole...","Prep Time: 30 mins, Cook Time: 25 mins, Total ...",https://www.allrecipes.com/thmb/6EpK8eOrDHd4MM...


In [9]:
# Extract the cuisine information from the 'cuisine_path' column by splitting the string using '/'
df_clean['cuisine'] = df_clean['cuisine_path'].apply(lambda x: x.split('/')[1])

In [10]:
# Count the occurrences of each unique value in the 'cuisine' column of a DataFrame 'df_clean'
df_clean['cuisine'].value_counts()

Desserts                          232
Side Dish                          81
Bread                              50
Cuisine                            48
Appetizers and Snacks              28
Main Dishes                        26
Meat and Poultry                   23
Breakfast and Brunch               20
Salad                              12
Soups, Stews and Chili Recipes      9
Seafood                             7
Drinks Recipes                      5
Everyday Cooking                    4
Quick Bread Recipes                 3
Sauces and Condiments               2
Soup Recipes                        2
Holidays and Events Recipes         2
Fruits and Vegetables               1
BBQ & Grilling                      1
Mexican                             1
Name: cuisine, dtype: int64

In [11]:
# Filter the 'df_clean' to select rows where the 'cuisine' column has the value 'Desserts'.
filt= df_clean['cuisine'] == 'Desserts'
df_dessert= df_clean[filt]

In [12]:
# Create DataFrames for each table
recipes_df = df_dessert[['id', 'recipe_name', 'prep_time', 'cook_time', 'total_time', 'servings', 'directions', 'url', 'cuisine', 'img_src']]
ingredients_df = df_dessert[['id', 'ingredients']]
ratings_df = df_dessert[['id', 'rating']]
nutritions_df = df_dessert[['id', 'nutrition']]

In [13]:
# Display the first few rows of the 'recipes_df' DataFrame
recipes_df.head()

,id,recipe_name,prep_time,cook_time,total_time,servings,directions,url,cuisine,img_src
1,1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,"Peel and core apples, then thinly slice. Set a...",https://www.allrecipes.com/recipe/12682/apple-...,Desserts,https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...
3,3,Apple Crisp,30 mins,45 mins,1 hrs 15 mins,12,Preheat the oven to 350 degrees F (175 degrees...,https://www.allrecipes.com/recipe/12409/apple-...,Desserts,https://www.allrecipes.com/thmb/uAzhPOh86PfR-N...
4,4,Apple Pie Filling,20 mins,20 mins,2 hrs 40 mins,40,Toss apples with lemon juice in a large bowl a...,https://www.allrecipes.com/recipe/12681/apple-...,Desserts,https://www.allrecipes.com/thmb/c0bbYaS1V_mTt_...
7,7,Apple-Cranberry Crisp,25 mins,40 mins,1 hrs 5 mins,8,Preheat the oven to 375 degrees F (190 degrees...,https://www.allrecipes.com/recipe/41893/apple-...,Desserts,https://www.allrecipes.com/thmb/MjDDFmBWiEx9f1...
10,10,Apple Turnovers,30 mins,25 mins,55 mins,8,Combine the lemon and 4 cups water in a large ...,https://www.allrecipes.com/recipe/59124/apple-...,Desserts,https://www.allrecipes.com/thmb/6EpK8eOrDHd4MM...


In [14]:
# Display the first few rows of the 'ingredients_df' DataFrame
ingredients_df.head()

,id,ingredients
1,1,"8 small Granny Smith apples, or as needed, ½ c..."
3,3,"10 cups all-purpose apples, peeled, cored and ..."
4,4,"18 cups thinly sliced apples, 3 tablespoons le..."
7,7,"2 pounds Granny Smith apples - peeled, cored a..."
10,10,"2 tablespoons lemon juice, 4 cups water, 4 Gr..."


In [15]:
# Split the 'ingredients' column by ',' and then explode it into separate rows
ingredients_df['ingredients'] = ingredients_df['ingredients'].str.split(',')
ingredients_df = ingredients_df.explode('ingredients')

# Rename the 'id' column to 'meal_id' in the 'ingredients_df' DataFrame
ingredients_df.rename(columns={'id': 'meal_id'}, inplace=True)

# Add a new 'id' column with values ranging from 1 to the length of the DataFrame
ingredients_df['id'] = range(1, len(ingredients_df) + 1)

In [16]:
# Select and reorder the columns
ingredients_df = ingredients_df[['id', 'meal_id', 'ingredients']]

# Display the first few rows of the updated DataFrame
ingredients_df.head()

,id,meal_id,ingredients
1,1,1,8 small Granny Smith apples
1,2,1,or as needed
1,3,1,½ cup unsalted butter
1,4,1,3 tablespoons all-purpose flour
1,5,1,½ cup white sugar


In [17]:
# pattern = r'\d+|\d+/\d+\b.*'
# ingredients_df['clean_ingredients']= ingredients_df['ingredients'].str.contains(pattern)

In [18]:
# ingredients_df.head(20)

In [19]:
# Display the first few rows of the 'ratings_df' DataFrame
ratings_df.head()

,id,rating
1,1,4.8
3,3,4.7
4,4,4.7
7,7,4.6
10,10,4.6


In [20]:
# Calculate the minimum and maximum ratings from the 'rating' column
r_min= ratings_df['rating'].min()
r_max= ratings_df['rating'].max()

print (r_min)
print(r_max)

2.0
5.0


In [21]:
# Rename the 'id' column to 'meal_id' in the 'ratings_df' DataFrame
ratings_df.rename(columns={'id': 'meal_id'}, inplace=True)

# Add a new 'id' column with values ranging from 1 to the length of the DataFrame
ratings_df['id'] = range(1, len(ratings_df) + 1)

# Select and reorder the columns you want to keep in the DataFrame
ratings_df = ratings_df[['id', 'meal_id', 'rating']]

# Display the first few rows of the updated DataFrame
ratings_df.head()

,id,meal_id,rating
1,1,1,4.8
3,2,3,4.7
4,3,4,4.7
7,4,7,4.6
10,5,10,4.6


In [22]:
# Display the first few rows of the 'nutritions_df' DataFrame
nutritions_df.head()

,id,nutrition
1,1,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole..."
3,3,"Total Fat 8g 11%, Saturated Fat 5g 25%, Choles..."
4,4,"Total Fat 0g 0%, Sodium 61mg 3%, Total Carbohy..."
7,7,"Total Fat 12g 15%, Saturated Fat 4g 22%, Chole..."
10,10,"Total Fat 26g 33%, Saturated Fat 8g 39%, Chole..."


In [23]:
# Extract and convert nutrition information into separate columns
nutritions_df['protein'] = nutritions_df['nutrition'].str.extract(r'Protein (\d+\.?\d*)g', expand=True).astype(float)
nutritions_df['carbohydrate'] = nutritions_df['nutrition'].str.extract(r'Total Carbohydrate (\d+\.?\d*)g', expand=True).astype(float)
nutritions_df['fat'] = nutritions_df['nutrition'].str.extract(r'Total Fat (\d+\.?\d*)g', expand=True).astype(float)
nutritions_df['dietary_fiber'] = nutritions_df['nutrition'].str.extract(r'Dietary Fiber (\d+\.?\d*)g', expand=True).astype(float)

In [24]:
# Drop the original 'nutrition' column as it is no longer needed
nutritions_df.drop(columns=['nutrition'], inplace=True)

In [25]:
# Rename the 'id' column to 'meal_id' in the 'nutritions_df' DataFrame
nutritions_df.rename(columns={'id': 'meal_id'}, inplace=True)

# Add a new 'id' column with values ranging from 1 to the length of the DataFrame
nutritions_df['id'] = range(1, len(nutritions_df) + 1)

# Select and reorder the columns you want to keep in the DataFrame
nutritions_df = nutritions_df[['id', 'meal_id', 'protein', 'carbohydrate', 'fat', 'dietary_fiber']]

# Display the first few rows of the updated DataFrame
nutritions_df.head()

,id,meal_id,protein,carbohydrate,fat,dietary_fiber
1,1,1,2.0,52.0,19.0,3.0
3,2,3,2.0,61.0,8.0,3.0
4,3,4,0.0,33.0,0.0,1.0
7,4,7,2.0,43.0,12.0,4.0
10,5,10,5.0,80.0,26.0,2.0


In [29]:
# Create a SQLAlchemy engine for connecting to a SQLite database named "recipes"
# engine = create_engine('sqlite:///Resources/DessertPaletteRecipesDB.db')

In [30]:
# # Write the 'recipes_df' DataFrame to an SQLite table named 'recipes'
# recipes_df.to_sql('recipes', engine, if_exists='append', index=False)

# # # Write the 'ingredients_df' DataFrame to an SQLite table named 'ingredients'
# ingredients_df.to_sql('ingredients', engine, if_exists='append', index=False)

# # # Write the 'ratings_df' DataFrame to an SQLite table named 'ratings'
# ratings_df.to_sql('ratings', engine, if_exists='append', index=False)

# # # Write the 'nutritions_df' DataFrame to an SQLite table named 'nutritions'
# nutritions_df.to_sql('nutritions', engine, if_exists='append', index=False)

232

In [31]:
# engine.dispose()